<a href="https://colab.research.google.com/github/likeaAI/pytorch/blob/main/test02_cloths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="V1xrnYqaT6VW3skv3nh8")
project = rf.workspace("example01").project("test02-l7ug9")
dataset = project.version(1).download("yolov5-obb")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 145 kB 5.2 MB/s 
     |████████████████████████████████| 178 kB 41.2 MB/s 
     |████████████████████████████████| 1.1 MB 55.2 MB/s 
     |████████████████████████████████| 67 kB 8.0 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 138 kB 76.4 MB/s 
     |████████████████████████████████| 596 kB 54.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for roboflow: filename=roboflow-0.2.4-py3-none-any.whl size=21356 sha256=78e8995907f7bb6ce068fd3874b5e4c42eb80b7cdcdd43ef8b024e46d37f661f
  Stored in directory: /root/.cache/pip/wheels/b7/2c/60/3733c0e323b1d7aec00f852264314bf6c48192c8e1dc92b3ee
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=8ad851938b606a6d09746f9d32709eeaaacfb353064ee29a2b45208aaa9d90ed
  Stored in directory: /root/.cache/pip/

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to test02-1 in yolov5-obb:: 100%|██████████| 778/778 [00:00<00:00, 1854.22it/s]


In [2]:
# yolov5 다운로드 
%cd /content
!git clone https://github.com/ultralytics/yolov5.git

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 12128, done.
remote: Total 12128 (delta 0), reused 0 (delta 0), pack-reused 12128
Receiving objects: 100% (12128/12128), 11.88 MiB | 29.40 MiB/s, done.
Resolving deltas: 100% (8399/8399), done.


In [ ]:
# content 폴더에 test02-1 폴더를 yolov5안으로 이동시킨다. 
 # content/yolov5/test02-1 

In [4]:
from glob import glob

train_img_list = glob('yolov5/test02-1/train/images/*.jpg')
val_img_list = glob('yolov5/test02-1/valid/images/*.jpg')


print(len(train_img_list) , len(val_img_list))

269 77


In [5]:
with open('yolov5/test02-1/train.txt' , 'w') as f : 
    f.write('\n'.join(train_img_list))
with open('yolov5/test02-1/val.txt' , 'w') as f : 
    f.write('\n'.join(val_img_list))

In [6]:
import yaml

with open('yolov5/test02-1/data.yaml' , 'r') as f : 
    data = yaml.safe_load(f) # yaml.load() x => yaml.safe_load(f)

print(data)


data['train'] = '/content/yolov5/test02-1/train.txt'   
data['val'] = '/content/yolov5/test02-1/val.txt'

with open ('yolov5/test02-1/data.yaml' , 'w') as f : 
    yaml.dump(data, f )

print(data)

{'path': '../datasets/roboflow', 'train': 'train/images', 'val': 'valid/images', 'test': 'test/images', 'nc': 4, 'names': ['jean', 'slack', 'top_long', 'top_short']}
{'path': '../datasets/roboflow', 'train': '/content/yolov5/test02-1/train.txt', 'val': '/content/yolov5/test02-1/val.txt', 'test': 'test/images', 'nc': 4, 'names': ['jean', 'slack', 'top_long', 'top_short']}


In [7]:
%cat /content/yolov5/test02-1/data.yaml

names:
- jean
- slack
- top_long
- top_short
nc: 4
path: ../datasets/roboflow
test: test/images
train: /content/yolov5/test02-1/train.txt
val: /content/yolov5/test02-1/val.txt


In [8]:
%cd /content/yolov5/

!python train.py --img 120 --batch 16 --epochs 50 \
--data /content/yolov5/test02-1/data.yaml \
--cfg ./models/yolov5s.yaml \
--weights yolov5s.pt \
--name test02_result

/content/yolov5
train: weights=yolov5s.pt, cfg=./models/yolov5s.yaml, data=/content/yolov5/test02-1/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=120, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=test02_result, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-236-gdcf8073 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, a

In [10]:
# 추론하기 

from IPython.display import Image
import os

val_img_path = val_img_list[0]

!python detect.py --weights /content/yolov5/runs/train/test02_result/weights/best.pt \

detect: weights=['/content/yolov5/runs/train/test02_result/weights/best.pt'], source=data/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-236-gdcf8073 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Traceback (most recent call last):
  File "detect.py", line 252, in <module>
    main(opt)
  File "detect.py", line 247, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 92, in run
    model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=half)
  File "/content/yolo